In [1]:
#SMILE LIKE STARRY NIGHT
import cv2
import numpy as np
import dlib
from math import hypot
from scipy.spatial import distance as dist
from cvzone.SelfiSegmentationModule import SelfiSegmentation
from scipy.interpolate import UnivariateSpline
# Loading Camera and Nose image and Creating mask
cap = cv2.VideoCapture(0)
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
BG = cv2.imread("bg.jpg",cv2.IMREAD_COLOR)
background = cv2.resize(BG,(640,480))
segmentor = SelfiSegmentation()
star_image = cv2.imread("star-01-removebg-preview.png",cv2.IMREAD_COLOR)
smile_image = cv2.imread("text-01-removebg-preview.png",cv2.IMREAD_COLOR)
_, frame = cap.read()
rows, cols, _ = frame.shape
nose_mask = np.zeros((rows, cols), np.uint8)
# Loading Face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
def LookupTable(x, y):
    spline = UnivariateSpline(x, y)
    return spline(range(256))
def Summer(img):
    increaseLookupTable = LookupTable([0, 64, 128, 256], [0, 80, 160, 256])
    decreaseLookupTable = LookupTable([0, 64, 128, 256], [0, 50, 100, 256])
    blue_channel, green_channel,red_channel  = cv2.split(img)
    red_channel = cv2.LUT(red_channel, increaseLookupTable).astype(np.uint8)
    blue_channel = cv2.LUT(blue_channel, decreaseLookupTable).astype(np.uint8)
    sum= cv2.merge((blue_channel, green_channel, red_channel ))
    return sum

In [3]:
while True:
    _, frame = cap.read()
    nose_mask.fill(0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(frame)
    Frames = frame
    #Frames = Summer(frame)
    for face in faces:
        landmarks = predictor(gray_frame, face)
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)
        nose_width = int(hypot(left_nose[0] - right_nose[0],
                           left_nose[1] - right_nose[1]) * 1.7) #icon to hơn mũi 1 chút
        nose_height = int(nose_width * 0.77) #0.77 là tỉ lệ chiều cao / chiều rộng thật của ảnh làm icon
        # New nose position
        top_left = (int(center_nose[0] - nose_width / 2),
                              int(center_nose[1] - nose_height / 2))
        bottom_right = (int(center_nose[0] + nose_width / 2),
                       int(center_nose[1] + nose_height / 2))
        # Adding the new nose
        nose_im = cv2.resize(star_image, (nose_width, nose_height))
        nose_im_gray = cv2.cvtColor(nose_im, cv2.COLOR_BGR2GRAY)
        _, nose_mask = cv2.threshold(nose_im_gray, 25, 255, cv2.THRESH_BINARY_INV)
        nose_area = frame[top_left[1]: top_left[1] + nose_height,
                    top_left[0]: top_left[0] + nose_width] #[cao,rộng]
        nose_area_no_nose = cv2.bitwise_and(nose_area, nose_area, mask=nose_mask)
        final_nose = cv2.add(nose_area_no_nose, nose_im)
        frame[top_left[1]: top_left[1] + nose_height,
                    top_left[0]: top_left[0] + nose_width] = final_nose
        #MOUTH IS SMILING
        left_lip = (landmarks.part(48).x, landmarks.part(48).y)
        right_lip = (landmarks.part(54).x, landmarks.part(54).y)
        center_lip = (landmarks.part(66).x, landmarks.part(66).y)
        lips_width = int(hypot(left_lip[0] - right_lip[0],
                           left_lip[1] - right_lip[1]) * 1.7)
        lips_height = int(lips_width * 0.77)

        # Calculate jaw width
        left_jaw = (landmarks.part(3).x, landmarks.part(3).y)
        right_jaw = (landmarks.part(13).x, landmarks.part(13).y)
        jaw_width = int(hypot(left_jaw[0] - right_jaw[0],
                           left_jaw[1] - right_jaw[1]) * 1.7)
        # Calculate the ratio of lips and jaw widths
        ratio = lips_width/jaw_width
        if ratio > 0.43 :
            # New smile position
            top_leftlip = (int(center_lip[0] - lips_width / 2),
                                int(center_lip[1] - lips_height / 2))
            bottom_rightlip = (int(center_lip[0] + lips_width / 2),
                        int(center_lip[1] + lips_height / 2))
            # Adding the smile
            lip_im = cv2.resize(smile_image, (lips_width, lips_height))
            lip_im_gray = cv2.cvtColor(lip_im, cv2.COLOR_BGR2GRAY)
            _, lip_mask = cv2.threshold(lip_im_gray, 25, 255, cv2.THRESH_BINARY_INV)
            lip_area = frame[top_leftlip[1]: top_leftlip[1] + lips_height,
                        top_leftlip[0]: top_leftlip[0] + lips_width]
            lip_area_no_lip = cv2.bitwise_and(lip_area, lip_area, mask=lip_mask)
            final_lip = cv2.add(lip_area_no_lip, lip_im)
            frame[top_leftlip[1]: top_leftlip[1] + lips_height,
                        top_leftlip[0]: top_leftlip[0] + lips_width] = final_lip
            Frames = Summer(frame)
    cv2.imshow("Frame",Frames)
    key = cv2.waitKey(1)
    if key == 27:
        break